In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
from ipyleaflet import Map, basemaps, WKTLayer, LayerGroup
from ipywidgets import Layout, HTML, Button, Box, VBox


In [2]:
plt.ioff() #avoids showing the plotting widget right here and now.
plt.rcParams["figure.figsize"] = (5,5)
center_map_coordinates = (25.0, 53.0)

m = Map(basemap=basemaps.Esri.WorldImagery, center=center_map_coordinates, zoom=7, 
       layout=Layout(width='500px', height='500px', 
                     max_width='500px',     
                     max_height='500px',
                     border='solid'))

fig = plt.figure()
fig.canvas.header_visible = False
fig.canvas.layout = Layout(#width='500px', height='500px', 
    max_width='500px', 
    max_height='500px',
    border='solid')
#fig.set_size_inches(4, 4)
graph = plt.plot([-1, -1, 1], [-1, 1, 1])

def position_string(i,j):
    dlat = 0.2
    dlon = 0.2
    lon = 53 + i*dlon
    lat = 25 + j*dlat
    return f"POLYGON(({lon} {lat}, {lon+dlon} {lat}, {lon+dlon} {lat+dlat}, {lon} {lat+dlat}, {lon} {lat}))"

def button_callback_generator(i, j, graph, fig):
    def call_back(btn):
        t = np.linspace(0, 10*np.pi, 1001)
        graph[0].set_data(np.sin((i+1)*t), np.sin((j+1)*t))
        plt.title(f"{i}, {j}", fontsize=16)
        fig.canvas.draw()
        fig.canvas.flush_events()
        #print(f"{i}, {j}")
    return call_back

wlayers = []
popups = []
for j in range(3):
    for i in range(7):
        wlayers.append(WKTLayer(wkt_string=position_string(i,j),
                                style={"weight": 1, 
                                       "color": "green",
                                       "fillOpacity": 0.1},
                                hover_style={"fillColor": "red",
                                             "fillOpacity": 0.7})
                      )
        message = HTML(value = f"Here! {i}, {j}")
        button = Button(
            description='Click me',
            disabled=False,
            icon='check' # (FontAwesome names without the `fa-` prefix)
        )
        button.on_click(button_callback_generator(i, j, graph, fig))
        popups.append(VBox([message, button]))
        wlayers[-1].popup = popups[-1]

virtual_reefs = LayerGroup(layers=wlayers)
m.add_layer(virtual_reefs);

#I still don't know how to layout correctly these widgets on the page.
box_layout = Layout(#display='flex',
                    flex_flow='row',
                    #align_items='stretch',
                    border='solid',
                    width='100%',
                    max_width='1050px')
b = Box(children=[m, fig.canvas], layout=box_layout)
display(b)
#display(fig.canvas)
#display(m)


Box(children=(Map(center=[25.0, 53.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_tit…